# Problem 3: The Game of Tic-Tac-Toe

Initial board (X to move):
```
__ | X  | __
O  | __ | __
X  | __ | O
```

## Evaluation Function

$$
\text{Eval}(s) = 8X_3 + 3X_2 + X_1 - (8O_3 + 3O_2 + O_1)
$$

Where:
- $X_3, X_2, X_1$ = Number of 3-in-a-row, 2-in-a-row, and 1-in-a-row for X
- $O_3, O_2, O_1$ = Number of 3-in-a-row, 2-in-a-row, and 1-in-a-row for O

---

## Approach

- Build the game tree until terminal states.
- Apply **Minimax**: X = MAX, O = MIN.
- Use **Alpha-Beta pruning** to reduce explored nodes.

---

## Results

**Possible first moves:** (1,1), (1,3), (2,2), (2,3), (3,2)

**Minimax values:**
- (1,1) → 0
- (1,3) → 5
- (2,2) → 5
- (2,3) → 0
- (3,2) → -8

**Best value = 5** → Optimal moves: (1,3) or (2,2)

**Nodes expanded:**
- Minimax = 226
- Alpha-Beta = 98 (≈57% fewer)

In [1]:
from copy import deepcopy

INITIAL_BOARD = [
    [None, 'X',   None],
    ['O',  None,  None],
    ['X',  None,  'O']
]

LINES = [
    # rows
    [(0,0),(0,1),(0,2)],
    [(1,0),(1,1),(1,2)],
    [(2,0),(2,1),(2,2)],
    # cols
    [(0,0),(1,0),(2,0)],
    [(0,1),(1,1),(2,1)],
    [(0,2),(1,2),(2,2)],
    # diags
    [(0,0),(1,1),(2,2)],
    [(0,2),(1,1),(2,0)],
]

def print_board(b):
    for r in b:
        print("|".join([(c if c is not None else "__") for c in r]))
    print()

def get_legal_moves(board):
    return [(r,c) for r in range(3) for c in range(3) if board[r][c] is None]

def apply_move(board, move, player):
    b2 = deepcopy(board)
    r,c = move
    b2[r][c] = player
    return b2

def winner(board):
    # return 'X' or 'O' or None
    for line in LINES:
        values = [board[r][c] for r,c in line]
        if values[0] is not None and values.count(values[0]) == 3:
            return values[0]
    return None

def is_terminal(board):
    if winner(board) is not None:
        return True
    if all(board[r][c] is not None for r in range(3) for c in range(3)):
        return True
    return False

def evaluation(board):
    # Evaluation(s) = 8*X3 + 3*X2 + X1  – (8*O3 + 3*O2 + O1)
    X3 = X2 = X1 = 0
    O3 = O2 = O1 = 0
    for line in LINES:
        vals = [board[r][c] for r,c in line]
        xcount = vals.count('X')
        ocount = vals.count('O')
        # ignore empty-only lines
        if ocount == 0 and xcount > 0:
            if xcount == 3:
                X3 += 1
            elif xcount == 2:
                X2 += 1
            elif xcount == 1:
                X1 += 1
        if xcount == 0 and ocount > 0:
            if ocount == 3:
                O3 += 1
            elif ocount == 2:
                O2 += 1
            elif ocount == 1:
                O1 += 1
    val = 8*X3 + 3*X2 + X1 - (8*O3 + 3*O2 + O1)
    return val

# --- Minimax (full expansion to terminals) ---
def minimax(board, player, stats):
    """
    stats is a dict to accumulate nodes visited: stats['nodes'] incremented on each call.
    Returns value (int).
    """
    stats['nodes'] += 1
    if is_terminal(board):
        return evaluation(board)
    if player == 'X':  # MAX
        best = float('-inf')
        for mv in get_legal_moves(board):
            val = minimax(apply_move(board, mv, 'X'), 'O', stats)
            if val > best:
                best = val
        return best
    else:  # MIN (O)
        best = float('inf')
        for mv in get_legal_moves(board):
            val = minimax(apply_move(board, mv, 'O'), 'X', stats)
            if val < best:
                best = val
        return best

# --- Alpha-Beta (full search with pruning) ---
def alphabeta(board, player, alpha, beta, stats):
    """
    stats is a dict with 'nodes' and 'prunes' counters.
    Returns value (int).
    """
    stats['nodes'] += 1
    if is_terminal(board):
        return evaluation(board)
    if player == 'X':
        value = float('-inf')
        for mv in get_legal_moves(board):
            value = max(value, alphabeta(apply_move(board, mv, 'X'), 'O', alpha, beta, stats))
            alpha = max(alpha, value)
            if alpha >= beta:
                stats['prunes'] += 1
                break  # beta cut-off
        return value
    else:
        value = float('inf')
        for mv in get_legal_moves(board):
            value = min(value, alphabeta(apply_move(board, mv, 'O'), 'X', alpha, beta, stats))
            beta = min(beta, value)
            if alpha >= beta:
                stats['prunes'] += 1
                break  # alpha cut-off
        return value

# Helper to compute minimax values for each first move
def evaluate_first_moves_minimax(board):
    moves = get_legal_moves(board)
    results = []
    for mv in moves:
        newb = apply_move(board, mv, 'X')
        stats = {'nodes': 0}
        val = minimax(newb, 'O', stats)
        results.append((mv, val, stats['nodes']))
    return results

# Helper to compute minimax values for each first move but using alphabeta (for comparison)
def evaluate_first_moves_alphabeta(board):
    moves = get_legal_moves(board)
    results = []
    for mv in moves:
        newb = apply_move(board, mv, 'X')
        stats = {'nodes': 0, 'prunes': 0}
        val = alphabeta(newb, 'O', float('-inf'), float('inf'), stats)
        results.append((mv, val, stats['nodes'], stats['prunes']))
    return results

# Run full minimax from the root with bookkeeping to also find the best move and count nodes:
def minimax_root(board):
    moves = get_legal_moves(board)
    best_val = float('-inf')
    best_moves = []
    total_nodes = 0
    per_move = []
    for mv in moves:
        newb = apply_move(board, mv, 'X')
        stats = {'nodes': 0}
        val = minimax(newb, 'O', stats)
        total_nodes += stats['nodes']
        per_move.append((mv, val, stats['nodes']))
        if val > best_val:
            best_val = val
            best_moves = [mv]
        elif val == best_val:
            best_moves.append(mv)
    return best_val, best_moves, per_move, total_nodes

# Run alpha-beta root-level search to find best move with pruning & counts
def alphabeta_root(board):
    moves = get_legal_moves(board)
    best_val = float('-inf')
    best_moves = []
    total_nodes = 0
    total_prunes = 0
    per_move = []
    # we'll evaluate each first move using alphabeta on the resulting board,
    # this also gives a sense of per-first-move nodes & prunes
    for mv in moves:
        newb = apply_move(board, mv, 'X')
        stats = {'nodes': 0, 'prunes': 0}
        val = alphabeta(newb, 'O', float('-inf'), float('inf'), stats)
        per_move.append((mv, val, stats['nodes'], stats['prunes']))
        total_nodes += stats['nodes']
        total_prunes += stats['prunes']
        if val > best_val:
            best_val = val
            best_moves = [mv]
        elif val == best_val:
            best_moves.append(mv)
    return best_val, best_moves, per_move, total_nodes, total_prunes

# --- Main execution and printing ---
if __name__ == "__main__":
    print("Initial board:")
    print_board(INITIAL_BOARD)

    # Minimax: evaluate every first move (X moves now)
    best_val_m, best_moves_m, per_move_m, total_nodes_m = minimax_root(INITIAL_BOARD)
    print("MINIMAX (full expansion) results:")
    for mv, val, nodes in per_move_m:
        print(f" Move {mv} (1-based: {(mv[0]+1,mv[1]+1)}) -> Minimax value = {val}, nodes expanded for this subtree = {nodes}")
    print(f" Best minimax value = {best_val_m}")
    print(" Best first-move(s):", [ (mv, (mv[0]+1,mv[1]+1)) for mv in best_moves_m ])
    print(f" Total nodes visited by full minimax (sum of subtree nodes) = {total_nodes_m}")
    print()

    # Alpha-Beta: evaluate every first move using alphabeta (to get per-move nodes/prunes)
    best_val_ab, best_moves_ab, per_move_ab, total_nodes_ab, total_prunes_ab = alphabeta_root(INITIAL_BOARD)
    print("ALPHA-BETA results (per-first-move alphabeta on resulting board):")
    for mv, val, nodes, prunes in per_move_ab:
        print(f" Move {mv} (1-based: {(mv[0]+1,mv[1]+1)}) -> AlphaBeta value = {val}, nodes = {nodes}, prunes = {prunes}")
    print(f" Best alpha-beta value = {best_val_ab}")
    print(" Best first-move(s):", [ (mv, (mv[0]+1,mv[1]+1)) for mv in best_moves_ab ])
    print(f" Total nodes visited by alpha-beta (sum of subtree nodes) = {total_nodes_ab}")
    print(f" Total prunes encountered across subtrees = {total_prunes_ab}")
    print()

    # For completeness: show the evaluation on the terminal states if we make one of the best moves
    print("Example: apply one optimal move and show some resulting board(s) and their evaluations.")
    example_mv = best_moves_m[0]
    b_after = apply_move(INITIAL_BOARD, example_mv, 'X')
    print(" After optimal move", example_mv, "(1-based:", (example_mv[0]+1, example_mv[1]+1), ")")
    print_board(b_after)
    # If this new board is terminal, show its evaluation; otherwise, show evaluation when it's finally terminal
    print("Evaluation of this board (by the heuristic):", evaluation(b_after))


Initial board:
__|X|__
O|__|__
X|__|O

MINIMAX (full expansion) results:
 Move (0, 0) (1-based: (1, 1)) -> Minimax value = 0, nodes expanded for this subtree = 47
 Move (0, 2) (1-based: (1, 3)) -> Minimax value = 5, nodes expanded for this subtree = 35
 Move (1, 1) (1-based: (2, 2)) -> Minimax value = 5, nodes expanded for this subtree = 39
 Move (1, 2) (1-based: (2, 3)) -> Minimax value = 0, nodes expanded for this subtree = 61
 Move (2, 1) (1-based: (3, 2)) -> Minimax value = -8, nodes expanded for this subtree = 43
 Best minimax value = 5
 Best first-move(s): [((0, 2), (1, 3)), ((1, 1), (2, 2))]
 Total nodes visited by full minimax (sum of subtree nodes) = 225

ALPHA-BETA results (per-first-move alphabeta on resulting board):
 Move (0, 0) (1-based: (1, 1)) -> AlphaBeta value = 0, nodes = 20, prunes = 6
 Move (0, 2) (1-based: (1, 3)) -> AlphaBeta value = 5, nodes = 17, prunes = 6
 Move (1, 1) (1-based: (2, 2)) -> AlphaBeta value = 5, nodes = 31, prunes = 6
 Move (1, 2) (1-based: (2, 